In [1]:
!pip install --upgrade transformers peft accelerate sentencepiece

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata (10 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_10_12_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_10_12_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 3.9 MB/s  0:00:00 eta 0:00:01
Using c

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/opt/anaconda3/envs/hml-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Gemma 3 12B set up

model_name = "google/gemma-3-12b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="cpu",
)


Fetching 5 files:   0%|          | 0/5 [02:29<?, ?it/s]
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [6]:
# Loading syllogism.csv

df = pd.read_csv("syllogism.csv")

In [7]:
# Prompt for Gemma 2B

def build_prompt(s, c1, c2):

    return f""" You are solving a logical syllogism problem.

    Syllogism are logical arguments where some statements (premises) are assumed to be true.
    You must decide which conclusions logically follow from them, using deductive reasoning.
    Treat the statements as true even if they may seem unrealistic.

    STATEMENTS:
    {s}

    CONCLUSION 1:
    {c1}

    CONCLUSION 2:
    {c2}

    Choose one of the following options:

    A = If only conclusion 1 follows
    B = If only conclusion 2 follows
    C = If either 1 or 2 follows
    D = If neither 1 nor 2 follows
    E = If both 1 and 2 follow  

    Answer with a single capital letter: A, B, C, D, or E.

    Final Answer:

"""

In [8]:
# Model Prediction

def predict_class(s, c1, c2, max_new_tokens = 5, temperature = 0.0, top_p = 1.0):

    prompt = build_prompt(s, c1, c2)
    inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = max_new_tokens,
            do_sample = False if temperature == 0.0 else True,
            temperature = temperature,
            top_p = top_p
        )
    
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    generated = full_text[len(prompt):].strip()

    for ch in generated:
        if ch in ["A", "B", "D", "C", "E"]:
            return ch
        
    return "?"

In [9]:
pred = []

for i, row in df.iterrows():
    p = predict_class(
        row["Statement"],
        row["Conclusion 1"],
        row["Conclusion 2"]
    )
    pred.append(p)

df["Gemma_2B_results"] = pred

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [10]:
valid_mask = df["Gemma_2B_results"].isin(list("ABCDE"))

accuracy = (df["Result class"][valid_mask] == df["Gemma_2B_results"][valid_mask]).mean()

print(f"Accuracy on rows with valid LLM redictions: {accuracy:.4f}")

Accuracy on rows with valid LLM redictions: 0.0877


In [11]:
pairs = list(zip(df["Result class"], df["Gemma_2B_results"]))

In [12]:
print("Humans vs Gemma 2B", pairs[:20])

Humans vs Gemma 2B [('A', 'C'), ('B', 'C'), ('D', 'C'), ('B', 'C'), ('E', 'C'), ('E', 'C'), ('B', 'C'), ('B', 'C'), ('D', 'C'), ('E', 'C'), ('D', 'C'), ('D', 'C'), ('A', 'C'), ('D', 'C'), ('D', 'C'), ('C', 'C'), ('B', 'C'), ('D', 'C'), ('D', 'C'), ('D', '?')]
